You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings

warnings.filterwarnings("ignore")

- Import from the crewAI libray.

In [3]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [4]:
import os

openai_api_key = os.environ["OPENAI_API_KEY"]
os.environ["OPENAI_MODEL_NAME"] = "gpt-3.5-turbo"


irom utils import get_openai_api_key

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [6]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
    "about the topic: {topic}."
    "You collect information that helps the "
    "audience learn something "
    "and make informed decisions. "
    "Your work is the basis for "
    "the Content Writer to write an article on this topic.",
    allow_delegation=False,
    verbose=True,
)

### Agent: Writer

In [8]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
    "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
    "a new opinion piece about the topic: {topic}. "
    "You base your writing on the work of "
    "the Content Planner, who provides an outline "
    "and relevant context about the topic. "
    "You follow the main objectives and "
    "direction of the outline, "
    "as provide by the Content Planner. "
    "You also provide objective and impartial insights "
    "and back them up with information "
    "provided by the Content Planner. "
    "You acknowledge in your opinion piece "
    "when your statements are opinions "
    "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
)

### Agent: Editor

In [9]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
    "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
    "from the Content Writer. "
    "Your goal is to review the blog post "
    "to ensure that it follows journalistic best practices,"
    "provides balanced viewpoints "
    "when providing opinions or assertions, "
    "and also avoids major controversial topics "
    "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [10]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
        "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
        "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
        "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
    "with an outline, audience analysis, "
    "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [11]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
        "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
        "3. Sections/Subtitles are properly named "
        "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
        "engaging introduction, insightful body, "
        "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
        "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
    "in markdown format, ready for publication, "
    "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [12]:
edit = Task(
    description=(
        "Proofread the given blog post for "
        "grammatical errors and "
        "alignment with the brand's voice."
    ),
    expected_output="A well-written blog post in markdown format, "
    "ready for publication, "
    "each section should have 2 or 3 paragraphs.",
    agent=editor,
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [13]:
crew = Crew(agents=[planner, writer, editor], tasks=[plan, write, edit], verbose=2)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [14]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: 

Title: The Latest Trends and Key Players in Artificial Intelligence

Outline:
I. Introduction
- Definition of Artificial Intelligence (AI)
- Importance of AI in various industries
- Preview of key trends and players in the AI space

II. Latest Trends in Artificial Intelligence
- Machine learning and deep learning advancements
- Natural language processing (NLP) innovations
- AI in healthcare, finance, and retail sectors
- Ethical considerations and AI regulation updates

III. Key Players in A

- Display the results of your execution as markdown in the notebook.

In [15]:
from IPython.display import Markdown

Markdown(result)

# The Latest Trends and Key Players in Artificial Intelligence

## Introduction

Artificial Intelligence (AI) has become an integral part of various industries, transforming business operations and enhancing technological capabilities. AI involves machines simulating human intelligence processes like learning, reasoning, and self-correction. Its significance in sectors such as healthcare, finance, and retail cannot be overstated, as it allows organizations to analyze data, automate tasks, and provide personalized customer experiences. This blog post will explore the latest trends and key players that are shaping the AI landscape, offering insights into the advancements and innovations propelling the industry forward.

## Latest Trends in Artificial Intelligence

An important trend in AI is the progress of machine learning and deep learning technologies. These advancements empower machines to learn from data, recognize patterns, and make decisions with minimal human intervention. Moreover, innovations in natural language processing (NLP) enable machines to comprehend and generate human language, resulting in the creation of virtual assistants and chatbots that enhance customer interactions. In industries like healthcare, finance, and retail, AI applications are revolutionizing operations, enhancing efficiency, and fostering growth. However, it is essential to consider ethical implications and stay updated on regulatory changes as AI technology evolves, ensuring responsible and ethical usage.

## Key Players in Artificial Intelligence

Major tech companies such as Google, Amazon, and Microsoft are at the forefront of AI research and development, utilizing their expertise and resources to drive innovation in the field. Startups are also disrupting the AI landscape by introducing new solutions and challenging traditional methods. Research institutions play a crucial role in advancing AI technologies through cutting-edge research and collaborations with industry partners. Understanding the key players in the AI sector enables professionals, decision-makers, students, and academics to remain informed about the latest developments and opportunities in the industry.

## Target Audience Analysis

Professionals in technology, business, and research sectors can benefit from staying abreast of AI trends and key players, gaining insights to inform strategic decisions and foster innovation within their organizations. Individuals interested in AI advancements and applications can explore the latest industry developments, discovering how AI is reshaping various sectors and impacting daily life. Decision-makers seeking insights on AI trends and key players can utilize this information to assess potential partnerships, investments, and opportunities in the AI landscape. Students and academics studying AI and related fields can enhance their knowledge and understanding of AI technologies, preparing for future careers in this rapidly evolving field.

## Conclusion

Artificial Intelligence is continuously evolving, with new trends and key players shaping the industry and driving innovation across diverse sectors. By remaining informed about the latest advancements, ethical considerations, and key players in the AI space, individuals can navigate the complexities of this dynamic industry and leverage AI's potential to drive growth and transformation. As AI revolutionizes the way we work, live, and interact with technology, it is crucial to stay updated on the latest trends and developments to make informed decisions and stay ahead of the curve. Subscribe to our newsletter for regular updates and insights on the latest trends and key players in Artificial Intelligence.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [16]:
topic = "Machine Learning Fundamental Terminology"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Machine Learning Fundamental Terminology.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Title: Understanding Machine Learning Fundamental Terminology: A Comprehensive Guide

Introduction:
- Brief explanation of what machine learning is
- Importance of understanding fundamental terminology in machine learning

Key Points:
1. Latest Trends in Machine Learning Fundamental Terminology:
- Overview of current trends in machine learning terminology
- Explanation of popular terms such as algorithms, models, and datasets

2. Key Players in Machine Learning Fundamental Terminology:
- Intr

In [17]:
Markdown(result)

# Understanding Machine Learning Fundamental Terminology: A Comprehensive Guide

Machine learning, a subset of artificial intelligence, is a rapidly evolving field that focuses on developing algorithms and models to enable computers to learn from data and make predictions or decisions. For individuals looking to explore this exciting field, understanding fundamental terminology is crucial as it lays the groundwork for more advanced concepts and applications.

Staying abreast of the latest trends in machine learning fundamental terminology is essential in today's dynamic landscape. Terms such as algorithms, models, and datasets are constantly evolving to meet industry demands. By keeping up with these trends, beginners and intermediate learners can navigate the complexities of machine learning terminology more effectively and stay ahead of the curve.

Influential individuals and organizations are key players in shaping the terminology landscape of machine learning. By understanding their contributions and perspectives, learners can gain valuable insights into the development and application of fundamental concepts in this field. Profiles of these key players offer context and inspiration for those aspiring to make a mark in the industry.

Recent developments in machine learning terminology provide valuable insights into industry trends and future directions. Analyzing and recapping these news items can deepen learners' understanding of the impact of new terminology and concepts. This knowledge equips them to anticipate future trends and stay informed in the ever-evolving world of machine learning fundamental terminology.

In conclusion, grasping machine learning fundamental terminology is essential for anyone venturing into artificial intelligence and data science. By compiling the latest trends, key players, and noteworthy news in this comprehensive guide, we aim to provide valuable insights and resources for our target audience of beginners and intermediate learners. Engage with the content, stay informed, and continue learning to unlock the full potential of machine learning fundamental terminology.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).